In [1]:
#NAME : MEHRAJ HARISH GAUD
#STUDENT ID : GAU20508418
#DATE : 24/03/2023

In [2]:
#IN THIS CELL WE IMPORT ALL THE LIBRARIES NEEDED.
#CREDITS: https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude
from geopy.geocoders import Nominatim
import geopy.distance
from queue import Queue
#CREDITS: https://levelup.gitconnected.com/python-3-ways-to-display-images-14e06ae1b33c
import cv2
from tabulate import tabulate
import time
import cowsay

In [12]:
def search_and_history():
    """IN THIS FUNCTION WE CALL THE ALGORITHM SEARCH FUNCTIONS ACCORDING TO THE USER INPUTS."""
    flag2 = 0
    total_distance = 0
    path = []
    search_heading = ["START LOCATION", "CITIES IN BETWEEN", "DESTINATION", "TOTAL DISTANCE (in mile)"]
    cities_to_travel = []
    table_elements = []
    path_table = []
    history_table = []
    history_heading = ["START LOCATION", "DESTINATION LOCATION", "DISTANCE TRAVELLED (in mile)","ALGORITHM USED"]
    history_table.append(history_heading)
    maps_nodes = {
        'Manchester': [('Liverpool', 40), ('York', 70), ("Newcastle", 140), ("Edinburgh", 220), ("Carlisle", 120)],
        'York': [('Newcastle', 80), ("Manchester", 70)],
        'Liverpool': [("Holyhead", 90), ("Manchester", 40)],
        'Holyhead' : [("Liverpool", 90)],
        'Carlisle': [('Glasgow', 100), ("Manchester", 120)],
        'Newcastle': [('Edinburgh', 110), ("Manchester", 140), ("York", 80)],
        'Edinburgh' : [("Glasgow", 50), ("Manchester", 220), ("Newcastle", 110)],
        'Glasgow' : [("Inverness", 170), ("Aberdeen", 140), ("Oban", 100), ("Edinburgh", 50), ("Carlisle", 120)],
        'Oban' : [("Inverness", 110),("Glasgow", 100)],
        'Aberdeen' : [("Inverness", 110), ("Glasgow", 140)],
        'Inverness' : [("Glasgow", 170), ("Oban", 110), ("Aberdeen",110)]
    }
    while flag2 == 0:
        flag = 0
        print("""\n       WELCOME TO SEARCH AND HISTORY SECTION OF PATH PILOT:
                 1. SEARCH
                 2. HISTORY
                 3. ENTER 0 TO EXIT""")
        choose = input("Enter your choice:")
        if choose == "1":
            while flag == 0:
                history = []
                flag1 = 0
                print("\n START SEARCHING HERE:")
                start_location = input("Enter your start location:")
                dest_location = input("Enter your destination location:")
                start = start_location.capitalize()
                destination = dest_location.capitalize()
                print("""\n       CHOOSE YOUR ALGORITHM TO BEGIN THE SEARCH:
                 1. A START SEARCH ALGORITHM
                 2. DFS SEARCH ALGORITHM
                 3. BFS SEARCH ALGORITHM""")
                choice = input("Enter your choice:")
                if choice == "1":
                    """************CALL ASTART FUCNTION***********"""
                    if start and destination in  maps_nodes:
                        algo_used = "A START"
                        path, total_distance = search_aStar(start, destination,maps_nodes)
                    else:
                        print("\nNO PATH BETWEEN THESE LOCATION ACCORDING TO GIVEN MAP!!!")

                elif choice == "2":
                    """****************CALL DFS*******************"""
                    if start and destination in  maps_nodes:
                        algo_used = "DFS"
                        path, total_distance = search_dfs(start, destination, maps_nodes)
                        for x in range(1, len(path)-1):
                            cities_to_travel.append(path[x])
                        combine_cities = ','.join(cities_to_travel)
                        path_table.append(search_heading)
                        table_elements.append(start)
                        table_elements.append(combine_cities)
                        table_elements.append(destination)
                        table_elements.append(total_distance)
                        path_table.append(table_elements)
                        print(tabulate(path_table, headers = 'firstrow', tablefmt = 'grid'))
                        path.clear()
                        cities_to_travel.clear()
                        table_elements.clear()
                        path_table.clear()
                    else:
                        print("\nNO PATH BETWEEN THESE LOCATION ACCORDING TO GIVEN MAP!!!")
                    

                elif choice == "3":
                    """*******************CALL BFS******************"""
                    if start and destination in  maps_nodes:
                        algo_used = "BFS"
                        path, total_distance = search_bfs(start, destination, maps_nodes)
                        for x in range(1, len(path)-1):
                            cities_to_travel.append(path[x])
                        combine_cities = ','.join(cities_to_travel)
                        path_table.append(search_heading)
                        table_elements.append(start)
                        table_elements.append(combine_cities)
                        table_elements.append(destination)
                        table_elements.append(total_distance)
                        path_table.append(table_elements)
                        print(tabulate(path_table, headers = 'firstrow', tablefmt = 'grid'))
                        path.clear()
                        cities_to_travel.clear()
                        table_elements.clear()
                        path_table.clear()
                    else:
                        print("\nNO PATH BETWEEN THESE LOCATION ACCORDING TO GIVEN MAP!!!")
                else:
                    print("\nPLEASE SELECT VALID ALGORITHM!")
                history.append(start)
                history.append(destination)
                history.append(total_distance)
                try:
                    history.append(algo_used)
                except UnboundLocalError:
                    continue
                history_table.append(history)
                while flag1 == 0:
                    search_again = input("Would like to search again?(yes/no)")
                    if search_again.lower() == "yes":
                        flag1 = 1
                    elif search_again.lower() == "no":
                        flag1 = 1
                        flag = 1
                    else:
                        print("PLEASE SELECT CORRECT OPTION!")
        elif choose == "2":
            print("\n THIS IS YOUR SEARCH HISTORY:")
            print(tabulate(history_table, headers = 'firstrow', tablefmt = 'fancy_grid'))
        elif choose == "0":
            flag2 = 1
        else:
            print("\nINVALID OPTION SELECTED")

In [4]:
def search_aStar(start_location, dest_location, maps_nodes):
    """THIS IS THE A STAR FUNCTION TO CALCULATE THE PATH AND DISTANCE BETWEEN START AND THE DESTINATION LOCATION.
    _____________________________________________________________________________________________________________
    INPUT : MAPS_NODES, START_LOCATION, DEST_LOCATION
    OUTPUT: PRINTS TRAVEL DETAILS IN TABLE FORMAT"""
    open_set = [start_location]
    closed_set = []
    heading = ["START LOCATION", "CITIES IN BETWEEN", "DESTINATION", "TOTAL DISTANCE (in mile)"]
    g = {} 
    parents = {}
    flag = 0
    g[start_location] = 0
    parents[start_location] = start_location 
    while len(open_set) > 0:
        n = None
        for v in open_set:
            if n == None or g[v] + heuristic_values(v, dest_location) < g[n] + heuristic_values(n, dest_location):
                n = v
        if n == dest_location or maps_nodes[n] == None:
            pass
        else:
            for (m, distance) in get_neighbors(n, maps_nodes):
                if m not in open_set and m not in closed_set:
                    open_set.append(m)
                    parents[m] = n
                    g[m] = g[n] + distance
                    
                else:
                    if g[m] > g[n] + distance:
                        g[m] = g[n] + distance
                        parents[m] = n          
                        if m in closed_set:
                            closed_set.remove(m)
                            open_set.append(m)
        if n == None:
            print('Path does not exist!')
            return None


        if n == dest_location:
            path = []
            cities_to_travel = []
            table_elements = []
            path_table = []
            while parents[n] != n:
                path.append(n)
                n = parents[n]
            path.append(start_location)
            path.reverse()
            flag = 1
            for i in range(1,len(path)-1):
                cities_to_travel.append(path[i])
            combine_cities = ','.join(cities_to_travel)
            table_elements.append(path[0])
            table_elements.append(combine_cities)
            table_elements.append(path[len(path) - 1])
            table_elements.append(str(g[path[len(path)-1]]))
            path_table.append(heading)
            path_table.append(table_elements)
            print(tabulate(path_table, headers = 'firstrow', tablefmt = 'grid'))
            return path, g[path[len(path)-1]]
            break
 
        open_set.remove(n)
        closed_set.append(n)
    if flag == 0:
        print('Path does not exist!')
        return None, None

In [5]:
def get_neighbors(v, maps_nodes):
    """THIS IS THE FUNCTION TO GET THE NEIGHBOR CITIY FROM THE CURRENT LOCATION.
    ____________________________________________________________________________
    INPUT : CURRENT LOCATION AND MAP
    OUTPUT: NEIGHBOR CITY"""
    if v in maps_nodes:
        return maps_nodes[v]
    else:
        return None

In [6]:
def heuristic_values(start_location, dest_location):
    """THIS FUCNTION IS USE TO CALCULATE THE HEURISTICS VALUES OF CITIES.
    _____________________________________________________________________
    INPUT : START LOCATION AND DESTINATION LOCATION
    OUTPUT: HEURISTICS VALUE"""
    H_dist = {
        start_location : st_dist(start_location, dest_location)
    }
    return H_dist[start_location]

In [7]:
def search_bfs(start_location, dest_location, maps_nodes):
    """THIS IS A BREADTH FIRST SEARCH FUCNTION TO CALCULATE DISTANCE AND PATH TO TRAVEL USING MAP, START AND DESTNATION LOCATION.
    _____________________________________________________________________________________________________________________________
    INPUT : MAP, START LOCATION, AND DESTINATION LOCATION
    OUTPUT: PATH AND DISTANCE"""
    visited = set() 
    Qcities = Queue()
    Qcities.put((start_location, [start_location], 0))
    while not Qcities.empty():
        current_location, path, distance = Qcities.get()
        if current_location == dest_location:
            return path, distance
        visited.add(current_location)
        for neighbor, neighbor_distance in maps_nodes[current_location]:
            if neighbor not in visited:
                Qcities.put((neighbor, path + [neighbor], distance + neighbor_distance))
    return None, None

In [8]:
def search_dfs(start_location, dest_location, maps_nodes):
    """THIS IS A DEPTH FOR SEARCH FUCNTION TO CALCULATE DISTANCE AND PATH TO TRAVEL USING MAP, START AND DESTNATION LOCATION.
    ___________________________________________________________________________________________________________________________
    INPUT : START LOCATION, DESTINATION LOCATION, AND MAP
    OUTPUT: PATH AND DISTANCE"""
    visited = set()
    path = []
    total_distance = 0
    def dfs_in(current_location):
        nonlocal visited, path, total_distance
        visited.add(current_location)
        path.append(current_location)
        if current_location == dest_location:
            return True
        for neighbor, dist in maps_nodes[current_location]:
            if neighbor not in visited:
                total_distance += dist
                if dfs_in(neighbor):
                    return True
                total_distance -= dist
        path.pop()
        return False
    dfs_in(start_location)
    if dest_location in visited:
        return path, total_distance
    return [], 0

In [9]:
def st_dist(start_location, dest_location):
    """THIS IS THE FUCNTION TO CALCULATE STRIAGHT LINE DISTANCE BETWEEN TWO CITIES USING LATITUDE AND LONGITUDE.
    ____________________________________________________________________________________________________________
    INPUT : START LOCATION AND DESTINATION LOCATION
    OUTPUT: STARIAGHT LINE DISTANCE"""
    city_coordinates = {
                        "Manchester": (53.4808, -2.2426),
                        "Newcastle": (54.9783, -1.6178),
                        "York": (53.9591, -1.0819),
                        "Carlisle": (54.8926, -2.9326),
                        "Edinburgh": (55.9533, -3.1883),
                        "Liverpool": (53.4084, -2.9916),
                        "Glasgow": (55.8642, -4.2518),
                        "Oban": (56.4139, -5.4706),
                        "Inverness": (57.4778, -4.2247),
                        "Aberdeen": (57.1497, -2.0943),
                        "Holyhead": (53.3081, -4.6318)
                    }
    location1 = city_coordinates[start_location]
    location2 = city_coordinates[dest_location]
    geolocator = Nominatim(user_agent = "Navigation system")
    distance = geopy.distance.geodesic(location1, location2).mi
    return distance

In [10]:
def about_navigation():
    """THIS IS THE FUNCTION TO DISPLAY HISTORY OF NAVIGATION SYSTEM."""
    cowsay.cow("WHAT IS THE HISTORY OF NAVIGATION SYSTEM?")
    print("HERE YOU GO:")
    point1 = "=> Navigation has been an essential part of human history, with early navigators using the stars and natural landmarks to   find their way."
    point2 = "=> In the early 20th century, radio navigation systems were introduced to help pilots navigate through fog and other adverse weather conditions."
    point3 = "=> In the 1960s, the Transit satellite system was launched, the first satellite-based navigation system that allowed ships  and aircraft to determine their location with unprecedented accuracy."
    point4 = "=> In the 1970s, the Omega navigation system was introduced, providing navigation assistance for ships and aircraft in areas where satellite navigation was not possible."
    point5 = "=> The 1980s saw the introduction of the Global Positioning System (GPS), which revolutionized navigation with its high     accuracy and global coverage."
    about = [point1, point2, point3, point4, point5]

    #We use flush and set it to true so that each character is printed to console without buffering.
    for i in range(len(about)):
        char = about[i]
        for x in char:
            #print(x)
            print(x, end='', flush=True)
            time.sleep(0.03)
        print()

In [13]:
def main_menu():
    """THIS IS THE MAIN PROGRAM TO COMPILE ALL THE FUNCTIONS ABOVE."""
    flag = 0
    while flag == 0:
        print("""\n       WELCOME TO PATH-PILOT NAVIGATION SYSTEM:
                 1. ABOUT US
                 2. OPEN MAP
                 3. SEARCH AND HISTORY
                 4. ENTER 0 TO EXIT""")
        choice = input("Enter your choice:")
        if choice == "1":
            about_navigation()
        elif choice == "2":
            img = cv2.imread("map.png", cv2.IMREAD_COLOR)
            cv2.imshow("Map", img)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
        elif choice == "3":
            search_and_history()
        elif choice == "0":
            flag = 1
        else:
            print("\nINVALID OPTION!")
main_menu()


       WELCOME TO PATH-PILOT NAVIGATION SYSTEM:
                 1. ABOUT US
                 2. OPEN MAP
                 3. SEARCH AND HISTORY
                 4. ENTER 0 TO EXIT
Enter your choice:3

       WELCOME TO SEARCH AND HISTORY SECTION OF PATH PILOT:
                 1. SEARCH
                 2. HISTORY
                 3. ENTER 0 TO EXIT
Enter your choice:1

 START SEARCHING HERE:
Enter your start location:Manchester
Enter your destination location:inverness

       CHOOSE YOUR ALGORITHM TO BEGIN THE SEARCH:
                 1. A START SEARCH ALGORITHM
                 2. DFS SEARCH ALGORITHM
                 3. BFS SEARCH ALGORITHM
Enter your choice:1
+------------------+---------------------+---------------+----------------------------+
| START LOCATION   | CITIES IN BETWEEN   | DESTINATION   |   TOTAL DISTANCE (in mile) |
+==================+=====================+===============+============================+
| Manchester       | Carlisle,Glasgow    | Inverness     |   

=> In the early 20th century, radio navigation systems were introduced to help pilots navigate through fog and other adverse weather conditions.
=> In the 1960s, the Transit satellite system was launched, the first satellite-based navigation system that allowed ships  and aircraft to determine their location with unprecedented accuracy.
=> In the 1970s, the Omega navigation system was introduced, providing navigation assistance for ships and aircraft in areas where satellite navigation was not possible.
=> The 1980s saw the introduction of the Global Positioning System (GPS), which revolutionized navigation with its high     accuracy and global coverage.

       WELCOME TO PATH-PILOT NAVIGATION SYSTEM:
                 1. ABOUT US
                 2. OPEN MAP
                 3. SEARCH AND HISTORY
                 4. ENTER 0 TO EXIT
Enter your choice:2

       WELCOME TO PATH-PILOT NAVIGATION SYSTEM:
                 1. ABOUT US
                 2. OPEN MAP
                 3. SEARCH A